In [1]:
%load_ext autoreload
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import csv
import json
import os 
import datetime
import matplotlib.pyplot as plt
import seaborn as sb
import shutil
from importlib import reload
from IPython.display import Image, display, HTML
import matplotlib.gridspec as gridspec
import mpld3
import pprint 
pp = pprint.PrettyPrinter(indent=4)

In [3]:
import config
import utils

import raw_log_types
import results
import tasks
import task_results
from main import VbsVis
import plot

from constants import *

In [4]:
%autoreload 2

In [5]:
VbsVis.flush_cache()
vbs = VbsVis()

# vbs.parse_team("SOMHunter", 
#                   ["sh-patrik", "sh-vit"], 
#                   verbose=True, 
#                   rewrite=False, 
#                   validate_fix=False,
#                   generate_DRES=False
#                  )
# vbs.parse_team("CollageHunter", 
#                   ["collage-jakub", "collage-premek"], 
#                   verbose=True,
#                   rewrite=False,  
#                   validate_fix=False, 
#                   generate_DRES=False
#                  )
# vbs.parse_team("LegacySOMHunter", 
#                   ["legacy-tereza", "legacy-franta-tomas"], 
#                   verbose=True, 
#                   rewrite=False, 
#                   validate_fix=False, 
#                   generate_DRES=False
#                  )

#VbsVis.help()

Error while deleting directory
%%% PARSING TASKS %%%
%%% Parsed 38 tasks. %%%
%%% DONE! %%%
%%% PARSING VERDICTS %%%
%%% DONE %%%


In [ ]:
#VbsVis.cache(vbs)

In [ ]:
def plot_timelines(self,tasks=None, teams=None, users=None, task_types=["V", "T"]):      
    types = set()

    # print("TIMEOUTS:")
    # print("(No response from the server for 30s, request timed out.)")

    # print("SERVER LAG:")
    # print("(Submit send before task end, processed after.)")

    for t in self.tasks().tasks():
        t_name = t.name()
        t_type = t.type()

        if ((tasks != None) and (not (t_name in tasks))):
            continue

        if not (t_type in task_types):
            print(t_type)
            continue

        print(f"\n%%%%%%%%%%%%%%%")
        print(f"%%% {t_name} %%%")
        print(f"%%%%%%%%%%%%%%%")
        
        t_dur = t.duration()
    
        xs_elapseds = {}
        ys_videos = {}
        ys_frames = {}
        ys_num_reported = {}
        ys_labels = {}
        
        submits_correct = {}
        submits_incorrect = {}
        submits_indeterminate = {}
        submits_timeout = {}
        submits_server_lag = {}
        submits_logged_out = {}
       
        # Flags
        flags = {}
        flags_changed = {}

        # Actions
        actions_times = {}
        actions_types = {}
        
        #
        # Fetch all arrays of results and submits for all tasks
        # 
        task_results = self.task_results().task_results_arrays()
        user_submits = self.task_results().task_submits_arrays()
        
        for team, team_dict in task_results.items():
            if ((teams != None) and (not (team in teams))):
                continue

            print(f"\t--- TEAM: {team} ---")

            # User names for this one plot
            this_users = [] 
            
            #
            # Generate array data
            #
            for user, user_results in team_dict.items():
                if ((users != None) and (not (user in users))):
                    continue

                print(f"\t\t--- {user} ---")
                this_users.append(user)

                user_task_results = user_results[t_name]
                user_task_submits = user_submits[team][user][t_name]
                
                # Input data
                xs_elapseds[user] = user_task_results.elapsed()
                ys_videos[user] = user_task_results.vid()
                ys_frames[user] = user_task_results.fr()
                flags[user] = user_task_results.c()
                flags_changed[user] = user_task_results.c_changed()
                ys_num_reported[user] = user_task_results.reported()

                ys_labels[user] = user_task_results.values()

                submit_xs_elapseds = user_task_submits.elapsed()
                submit_positions, submit_reported = utils.find_submit_positions(submit_xs_elapseds, xs_elapseds[user], ys_videos[user], ys_num_reported[user])

                submit_positions = list(map(lambda x: x if x != None else config.avs_pos(), submit_positions))
                submits = list(zip(submit_xs_elapseds,submit_positions, user_task_submits.c(), submit_reported))
                
                submits_correct[user] = list(map(lambda y: (y[0], y[1], y[3]), filter(lambda x: x[2] == "T", submits)))
                submits_incorrect[user] = list(map(lambda y: (y[0], y[1], y[3]), filter(lambda x: x[2] == "F", submits)))
                submits_indeterminate[user] = list(map(lambda y: (y[0], y[1], y[3]), filter(lambda x: x[2] == "I", submits)))
                submits_timeout[user] = list(map(lambda y: (y[0], y[1], y[3]), filter(lambda x: x[2] == "TIMEOUT", submits)))
                submits_server_lag[user] = list(map(lambda y: (y[0], y[1], y[3]), filter(lambda x: x[2] == "SERVER_LAG", submits)))
                submits_logged_out[user] = list(map(lambda y: (y[0], y[1], y[3]), filter(lambda x: x[2] == "LOGGED_OUT", submits)))

                # print(f"TIMEOUTS: >> {len(submits_timeout[user])} <<")
                # print(f"SERVER LAG: >> {len(submits_server_lag[user])} <<")

                actions_times[user], actions_types[user] = self.task_actions_array(team, user, t_name)
                
                # pp.pprint(submits_correct)
                # pp.pprint(submits_incorrect)
                
            # Canvas size
            plt.rcParams["font.family"] = "stixgeneral"
            plt.rcParams["mathtext.fontset"] = "stix"
            fig = plt.figure(figsize=(16,6))
            
            gs_inner = fig.add_gridspec(2, 1, hspace=0.15, height_ratios=[10,1])
            ax = fig.add_subplot(gs_inner[0, 0])
           

            # Title
            ax.set_title(f"{team}: {t_name}", y=1.0, fontsize=40, loc='right')
            
            #ax.yaxis.set_visible(False)
            # ax.spines['right'].set_visible(True)
            # ax.spines['left'].set_visible(True)
            # ax.spines['top'].set_visible(True)
            ax.xaxis.set_ticks_position('bottom')

            # Set y to log scale
            ax.set_yscale("log")
            # Set y axis range
            ax.set_ylim(1, 100000)
            ax.set_xlim(xmin=0, xmax=450)
            ax.set_xticks([])
            ax.set_yticks([1,10,100,1000,10000,100000])
            # ax.yaxis.grid(True)
            ax.xaxis.grid(True)

            #ax.grid(which='both')
            ax.grid()
            
            ax.tick_params(axis='x', labelsize=20)
            ax.tick_params(axis='y', labelsize=20)

            for user_idx, member_ID in enumerate(this_users):
                ##########################
                # Draw video & frame pos
                ##########################

                #
                # Get rid of None values for AVS
                #
                ys_videos[member_ID] = list(map(lambda x: x if (x != None) else config.avs_pos(), ys_videos[member_ID]))
                ys_frames[member_ID] = list(map(lambda x: x if (x != None) else config.avs_pos(), ys_frames[member_ID]))

                #
                # Video positions
                #
                ax.step(xs_elapseds[member_ID], ys_videos[member_ID], where='post', label='post', color=COLORS[user_idx] , linewidth="2", alpha=0.7)

                #
                # Frame positions
                # (just horizontal lines)
                fst = xs_elapseds[member_ID][:-1]
                snd = xs_elapseds[member_ID][1:]
                ys = ys_frames[member_ID]
                zipped = list(zip(fst, snd, ys))
                rrr = list(filter(lambda x: x[2] < 20000, zipped))

                if len(rrr) != 0:
                    fst, snd, ys = list(zip(*rrr))
                    #plt.step(xs_elapseds[member_ID], ys_frames[member_ID], where='post', label='post', color=COLORS[user_idx], linestyle=':', linewidth="4")
                    ax.hlines(ys, fst, snd, label='post', color=COLORS[user_idx], linestyle=':', linewidth=3, alpha=0.7)

                #
                # Limit of send results
                #
                ax.step(xs_elapseds[member_ID], ys_num_reported[member_ID], where='post', label='post', color=COLORS[user_idx], linestyle='dashdot', linewidth="1")

                ##########################
                # Draw result types
                ##########################
                #print("---")
                # pp.pprint(flags[member_ID])
                # pp.pprint(flags_changed[member_ID])
                vbs.print_task_course([team], [member_ID], [t_name], events=["r"])
                
                for iii, (fs, fsc, l) in enumerate(list(zip(flags[member_ID], flags_changed[member_ID], ys_labels[member_ID]))):

                    x = xs_elapseds[member_ID][iii]
                    y = ys_videos[member_ID][iii]
                    color = COLORS[user_idx]
                    color_c = COLORS_CHAHGE[user_idx] # todo change
                    
                    is_initial, is_nn, is_text, is_bitmap, is_like, is_temporal, is_reloc, is_canvas, is_filter = config.classes_to_flags(fs)
                    initial_ch, nn_ch, text_ch, bitmap_ch, like_ch, temporal_ch, reloc_ch, canvas_ch, filter_ch = config.classes_to_changed_flags(fsc)
                    
                    # print(f"bitmap_ch: {bitmap_ch}")
                    # print(f"reloc_ch: {reloc_ch}")
                    # - SHAPE -
                    if is_reloc:
                        plot.plot_relocation_mark(ax, x, y, reloc_ch, color, color_c)
                    if is_canvas:
                        plot.plot_canvas_mark(ax, x, y, canvas_ch, color, color_c)

                    # - LETTER -
                    if is_temporal:
                        plot.plot_temporal_mark(ax, x, y, temporal_ch, color, color_c)
                    if is_like:
                        plot.plot_like_mark(ax, x, y, like_ch, color, color_c)

                    # - MAIN -
                    if is_initial:
                        plot.plot_initial_mark(ax, x, y, initial_ch, color, color_c)
                    if is_text:
                        plot.plot_text_mark(ax, x, y, text_ch, color, color_c)
                    if is_bitmap:
                        plot.plot_bitmap_mark(ax, x, y, bitmap_ch, color, color_c)
                    if is_nn:
                        plot.plot_nn_mark(ax, x, y, nn_ch, color, color_c)
                    
                #print("---")
                ##########################
                # Draw the end of the task
                ##########################
                ax.axvline(x=t_dur, color='r', linestyle='-', linewidth="5")

                ##########################
                # Draw submits
                ##########################
                font_dict = {
                    'family': 'serif',
                    'color':  'red',
                    'weight': 'bold',
                    'size': 16,
                }
                submit_arrays = [
                    (submits_correct, "", "green", "<|-,head_width=0.4,head_length=.4", 10, 2, None, 1),
                    (submits_incorrect, "", "red", "]-,widthA=0.3, lengthA=0.2", 10, 2, None, 2),
                    (submits_indeterminate, "indeter", "grey", "-", 10, 2, None, 3),
                    (submits_logged_out, "loged out", "red", "-", 10, 2, font_dict, 3),
                    (submits_timeout, "timeout", "red", "-", 10, 2, font_dict, 3),
                    (submits_server_lag, "server lag", "red", "-", 10, 2, font_dict, 3)
                ]

                #
                # Iterate over all the submit types
                #
                for arr, t, col, arrow, size, lw, font, off in submit_arrays:
                    for time, pos, reported in arr[member_ID]:
                        #plt.axvline(x=s, color='g', linewidth="2") ## Line only
                        xy_text = (40, 40) # Where the text will be, it determines the arrow direction
                        #
                        # If out of the reported range
                        #
                        if (pos >= reported):
                            pos = reported
                            xy_text = (xy_text[0], -xy_text[1])


                        ax.annotate("", xy=(time, pos), xytext=xy_text, textcoords='offset points', rotation=90, size=size,
                            horizontalalignment='left', verticalalignment="center", 
                            color=COLORS[user_idx] if t!="" else col, 
                            arrowprops=dict(
                                arrowstyle=arrow, 
                                alpha=0.4 if t!="" else 1,
                                facecolor=COLORS[user_idx]if t!="" else col, 
                                edgecolor=COLORS[user_idx]if t!="" else col,
                                lw=lw,
                            )
                        )

                        ax.text(time + (450 * 0.04), pos + (pos * 9) * (off / 2), t, size=12, rotation=90,
                            ha="center", va="bottom", color=col, fontdict=font
                        )

                        #
                        # For non-AVS task fade out the rest of the timeline
                        #
                        if (t_type != "A" and t == "correct"):
                            ax.axvspan(time, 450, facecolor='0.05', alpha=0.15)
                                   
                
                #
                # #######################
                #   Event plot
                # #######################
                #

                ax2 = fig.add_subplot(gs_inner[1,0])
                ax2.set_xlim([0, 450])
                ax2.set_yticks([])
                ax2.tick_params(axis='x', labelsize=20)
                ax2.tick_params(axis='y', labelsize=20)

                
                ##########################
                # Draw actions xoxo
                ##########################
                # pp.pprint(actions_times[user])
                # pp.pprint(actions_types[user])
                for iii, (time, tpe) in enumerate(list(zip(actions_times[member_ID], actions_types[member_ID]))):
                    x = time

                    types.add(tpe)
                    y = config.action_type_to_y(tpe)

                    color = COLORS[user_idx]
                    color_c = COLORS[user_idx] # todo change
                    line_style = "-"
                    plt.vlines(x=x, ymin=0, ymax=1, color=color, label="...", linestyle=line_style, alpha=0.5)

            #ax.get_yaxis().set_ticklabels([])
            plt.tight_layout()
            plt.savefig(os.path.join(config.out_dir("timelines", team), t_name + ".pdf"))
            for u in this_users:
                acts = vbs.task_course(team, u, t_name, events=["r", "a", "s"])
                with open(os.path.join(config.out_dir("timelines", team), t_name + "-" + u + "-actions.txt"),"w") as ofs:
                    for s in acts:
                        ofs.write(s[1].__str__())

            plt.show()

    print(types)

In [ ]:
#plot_timelines(vbs, tasks=["27_v21-14"])
plot_timelines(vbs, task_types=["T", "V"])


# Info queries

In [ ]:
#vbs.print_task_course(["SOMHunter"], ["sh-patrik"], ["09_a21-7"])

In [6]:
vbs.print_tasks()

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

#vbs.print_tasks(["01_v21-1"])

***############################***
***###*** VBS 2021 TASKS ***###***
***############################***
>>>>>>>>>>>>> STATS <<<<<<<<<<<<<<<<
Number of tasks: 38
Unique video IDs: 
88, 1162, 1939, 2034, 2224, 2464, 2519, 3085, 3317, 3331, 3870, 3885, 4035, 4178, 4225, 4316, 4888, 4979, 5146, 5531, 6228, 6246, 6561, 6827, 6979, 7116, 7421, 
>>>>>>>>>>>>> TASKS <<<<<<<<<<<<<<<<
	*** Task ***
	name: 01_v21-1
	type: V
	timestamps: (1624277458765, 1624277758765)
	times: (21-06-2021 14:10:58, 21-06-2021 14:15:58)
	text: None
	target: *** TaskTarget ***
		video_ID: 4178
		interval: (1875.0, 2300.0)




------------------------------------
	*** Task ***
	name: 02_a21-5
	type: A
	timestamps: (1624277828154, 1624278128154)
	times: (21-06-2021 14:17:08, 21-06-2021 14:22:08)
	text: Find shots of a person holding or waving a flag.
	target: None

------------------------------------
	*** Task ***
	name: 03_t21-1
	type: T
	timestamps: (1624278204021, 1624278624021)
	times: (21-06-2021 14:23:24, 21-06-2021 14:30:24)
	text: Graffity artists spraying purple outline of the letters RTH. Letters are white/green on a wall with a blueish graffity on the left. He wears a blue jacket over a green hoodie.

	target: *** TaskTarget ***
		video_ID: 3085
		interval: (7850.0, 8425.0)




------------------------------------
	*** Task ***
	name: 04_v21-2
	type: V
	timestamps: (1624278710026, 1624279010026)
	times: (21-06-2021 14:31:50, 21-06-2021 14:36:50)
	text: None
	target: *** TaskTarget ***
		video_ID: 1162
		interval: (23050.0, 23700.0)




------------------------------------
	*** Task ***
	name: 05_a21-9
	type: A
	timestamps: (1624279149067, 1624279449067)
	times: (21-06-2021 14:39:09, 21-06-2021 14:44:09)
	text: Find shots of at least one person drinking beer.
	target: None

------------------------------------
	*** Task ***
	name: 06_t21-2
	type: T
	timestamps: (1624279524639, 1624279944639)
	times: (21-06-2021 14:45:24, 21-06-2021 14:52:24)
	text: A hand opening and closing a window of a mountain hut. There are snow covered mountains outside. The weather is sunny, the shadow of the hut is visible in the snow.

	target: *** TaskTarget ***
		video_ID: 2224
		interval: (2097.9021000000002, 2337.6623400000003)




------------------------------------
	*** Task ***
	name: 07_v21-3
	type: V
	timestamps: (1624280086318, 1624280386318)
	times: (21-06-2021 14:54:46, 21-06-2021 14:59:46)
	text: None
	target: *** TaskTarget ***
		video_ID: 5146
		interval: (179.82018, 779.22078)




...

------------------------------------
	*** Task ***
	name: 33_a21-11
	type: A
	timestamps: (1624448946650, 1624449246650)
	times: (23-06-2021 13:49:06, 23-06-2021 13:54:06)
	text: Find shots of people skiing, shot with the camera looking into the sun (back-lit shot, possibly with lens flare).
	target: None

------------------------------------
	*** Task ***
	name: 34_v21-18
	type: V
	timestamps: (1624449356322, 1624449656322)
	times: (23-06-2021 13:55:56, 23-06-2021 14:00:56)
	text: None
	target: *** TaskTarget ***
		video_ID: 5531
		interval: (1850.0, 2175.0)




------------------------------------
	*** Task ***
	name: 35_v21-19
	type: V
	timestamps: (1624450582353, 1624450882353)
	times: (23-06-2021 14:16:22, 23-06-2021 14:21:22)
	text: None
	target: *** TaskTarget ***
		video_ID: 2464
		interval: (5825.0, 6325.0)




------------------------------------
	*** Task ***
	name: 36_v21-20
	type: V
	timestamps: (1624451061608, 1624451361608)
	times: (23-06-2021 14:24:21, 23-06-2021 14:29:21)
	text: None
	target: *** TaskTarget ***
		video_ID: 6827
		interval: (4531.464, 4891.103999999999)




------------------------------------
	*** Task ***
	name: 37_a21-6
	type: A
	timestamps: (1624451557507, 1624451857507)
	times: (23-06-2021 14:32:37, 23-06-2021 14:37:37)
	text: Find underwater shots of one or more fish.
	target: None

------------------------------------
	*** Task ***
	name: 38_v21-21
	type: V
	timestamps: (1624451948839, 1624452248839)
	times: (23-06-2021 14:39:08, 23-06-2021 14:44:08)
	text: None
	target: *** TaskTarget ***
		video_ID: 3885
		interval: (6458.541465, 6863.13687)




------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [ ]:
vbs.print_tool_features()

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

vbs.print_tool_features(["SOMHunter"])

In [ ]:
vbs.print_task_results_arrays()

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

vbs.print_task_results(["SOMHunter"], ["sh-patrik", "sh-vit"], ["09_a21-7"])

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
vbs.print_task_results()

In [ ]:
#
# Print results from the given interval
#
vbs.print_results("SOMHunter", "sh-patrik", 1624281095339, 1624281395339)
vbs.print_results("SOMHunter", "sh-vit", 1624281095339, 1624281395339)


In [ ]:
#
# Print submits for each task
#
vbs.print_task_submits_arrays()
vbs.print_task_submits()

In [ ]:
vbs.print_task_course(["SOMHunter"], ["sh-patrik"], ["27_v21-14"], events=["r"])
